In [73]:
import torch
import torch.nn as nn
import numpy as np
from torchview import draw_graph

device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [74]:
import random



def seed_everything(seed: int) -> None:
    """Seeds everything so that experiments are deterministic.

    Args:
        seed (int): Seed value.
    """

    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(134)

In [75]:
sequence = 30
features = 12
learning_rate = 0.001
batch_size = 64
num_class = 10
num_epoch = 10

# classes = ['aimchat', 
#            'email', 
#            'facebookChat', 
#            'ftps',
#            'gmailchat', 
#            'hangoutaudio', 
#            'icqchat', 
#            'netflix', 
#            'scp', 
#            'sftp', 
#            'skypevideo', 
#            'spotify', 
#            'vimeo', 
#            'voipbuster', 
#            'youtube']

classes = ('AIM Chat','Email','Facebook Audio','Facebook Chat','Gmail Chat','Hangouts Chat','ICQ Chat','Netflix','Spotify','Youtube')

In [76]:
# x_train = np.load('x_train_30K.npy')
# y_train = np.load('y_train_30K.npy')
# x_test = np.load('x_test_30K.npy')
# y_test = np.load('y_test_30K.npy')

x_train = np.load("x_train-MLP-Multiclass-ISCX-740features.npy")
y_train = np.load("y_train-MLP-Multiclass-ISCX-740features.npy")
x_test = np.load("x_test-MLP-Multiclass-ISCX-740features.npy")
y_test = np.load("y_test-MLP-Multiclass-ISCX-740features.npy")

In [77]:
x_train = np.delete(x_train, [12,13,14,15,16,17,18,19], 1)
x_test = np.delete(x_test, [12,13,14,15,16,17,18,19], 1)

x_train = x_train[:,:sequence*features]
x_test = x_test[:,:sequence*features]

# x_train = torch.from_numpy(x_train)
# y_train = torch.from_numpy(y_train)
# x_test = torch.from_numpy(x_test)
# y_test = torch.from_numpy(y_test)

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).float()

print(x_train.shape)
print(x_train.shape)

torch.Size([234964, 360])
torch.Size([234964, 360])


In [78]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)


train_loader = torch.utils.data.DataLoader(train_dataset,
                                           shuffle=True,
                                           batch_size=batch_size)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                           shuffle=False,
                                           batch_size=batch_size)

In [79]:
import gc
del train_dataset, test_dataset, x_train, y_train, x_test, y_test
gc.collect()

3827

In [84]:
class NtCNN(nn.Module):
    def __init__(self):
        super(NtCNN, self).__init__()

        # 1x1 convolution branch
        self.branch1x1 = nn.Sequential(
            nn.Conv1d(sequence, 16, kernel_size=1),
            nn.ReLU()
        )

        # 1x1 convolution followed by 3x3 convolution branch
        self.branch3x3 = nn.Sequential(
            nn.Conv1d(sequence, 24, kernel_size=1),  # Reduce channels from 36 to 24
            nn.Conv1d(24, 32, kernel_size=3, padding=1)  # 3x3 convolution 
            ,nn.ReLU()
            #,nn.GELU()    
        )

        # 1x1 convolution followed by 5x5 convolution branch
        self.branch5x5 = nn.Sequential(
            nn.Conv1d(sequence, 8, kernel_size=1),  # Reduce channels from 36 to 8
            nn.Conv1d(8, 16, kernel_size=5, padding=2)  # 5x5
            ,nn.ReLU()
            #,nn.GELU()            
        )

        # 3x3 max pooling followed by 1x1 convolution branch
        self.branch_pool = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
            nn.Conv1d(sequence, 16, kernel_size=1)  # Reduce channels to 16
            ,nn.ReLU()
            #,nn.GELU()  
        )

        self.fc1 = nn.Linear(80*12, 128)
        self.activation5 = nn.ReLU()

        self.fc2 = nn.Linear(128, 64)
        self.activation6 = nn.ReLU()
        
        self.fc3 = nn.Linear(64,num_class)
        
    def forward(self, x):
        
        x = x.view(-1, sequence, features)
        
        branch1x1 = self.branch1x1(x)
        branch3x3 = self.branch3x3(x)
        branch5x5 = self.branch5x5(x)
        branch_pool = self.branch_pool(x)
        outputs = [branch1x1, branch3x3, branch5x5, branch_pool]
        x = torch.cat(outputs, 1)
        #print(x.shape)
        x = x.view(-1, 80*12)
        # #print(x.shape)
        x = self.fc1(x)
        x = self.activation5(x)
        # #print(x.shape)
        x = self.fc2(x)
        x = self.activation6(x)
        
        x = self.fc3(x)
        # #print(x.shape)
        return x



model = NtCNN().to(device)

from torchinfo import summary
summary(model, 
        input_size=[batch_size, sequence, features], 
        device=device, 
        col_names=["input_size","output_size", "num_params"])

# architecture = 'InceptionBlock'
# model_graph = draw_graph(model, input_size=(batch_size, features, timestep), graph_dir ='TB' , roll=True, expand_nested=True, graph_name=f'self_{architecture}',save_graph=True,filename=f'self_{architecture}')
# model_graph.visual_graph

        

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NtCNN                                    [64, 30, 12]              [64, 10]                  --
├─Sequential: 1-1                        [64, 30, 12]              [64, 16, 12]              --
│    └─Conv1d: 2-1                       [64, 30, 12]              [64, 16, 12]              496
│    └─ReLU: 2-2                         [64, 16, 12]              [64, 16, 12]              --
├─Sequential: 1-2                        [64, 30, 12]              [64, 32, 12]              --
│    └─Conv1d: 2-3                       [64, 30, 12]              [64, 24, 12]              744
│    └─Conv1d: 2-4                       [64, 24, 12]              [64, 32, 12]              2,336
│    └─ReLU: 2-5                         [64, 32, 12]              [64, 32, 12]              --
├─Sequential: 1-3                        [64, 30, 12]              [64, 16, 12]              --
│    └─Conv1d: 2-6            

178.99322902329732 80.51555729965912
Epoch : 1/10, Loss: 0.0487, Validation Loss: 0.0512
100%
 3672/3672 [00:26<00:00, 155.65it/s]
166.9994734448701 75.575640315481
Epoch : 2/10, Loss: 0.0455, Validation Loss: 0.0480

In [85]:
from tqdm.auto import tqdm
import time
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# import wandb

# wandb.init(project="torch-CNN",
#            #mode='offline',
#            config = {"learning_rate": learning_rate, "epochs": num_epoch, "batch_size": batch_size})

# wandb.watch(model, criterion=criterion, log='all', log_freq=100)

start = time.time()
for epoch in range(num_epoch):
    avgloss = 0
    for i, (sample, labels) in enumerate(tqdm(train_loader)):
        model.train()
        sample = sample.to(device)
        labels = labels.to(device)

        predictions = model(sample)
        
        loss = criterion(predictions,labels)

        avgloss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        
    avg_val_loss = 0

    model.eval()
    with torch.inference_mode():
        for samples, labels in test_loader:
            samples = samples.to(device)
            labels = labels.to(device)

            predictions = model(samples)
            #labels = torch.argmax(labels, dim=1)
            
            val_loss = criterion(predictions, labels)

            avg_val_loss += val_loss.item()

    
    avgloss /= len(train_loader)
    avg_val_loss /= len(test_loader)
    print(f'Epoch : {epoch+1}/{num_epoch}, Loss: {avgloss:.4f}, Validation Loss: {avg_val_loss:.4f}')
    # wandb.log({"epoch": (epoch+1), "loss": avgloss, "Validation Loss": avg_val_loss})
end = time.time()

print(f'Time Taken:', (end-start))

  0%|          | 0/3672 [00:00<?, ?it/s]

3047.541883856058 822.7246540486813
Epoch : 1/10, Loss: 0.8299, Validation Loss: 0.5227


  0%|          | 0/3672 [00:00<?, ?it/s]

1591.1087301820517 576.6133679077029
Epoch : 2/10, Loss: 0.4333, Validation Loss: 0.3663


  0%|          | 0/3672 [00:00<?, ?it/s]

...

411.9713514104951 183.1888794163242
Epoch : 8/10, Loss: 0.1122, Validation Loss: 0.1164


  0%|          | 0/3672 [00:00<?, ?it/s]

346.82905289984774 145.1801425931044
Epoch : 9/10, Loss: 0.0945, Validation Loss: 0.0922


  0%|          | 0/3672 [00:00<?, ?it/s]

301.4989258693531 135.1786576739978
Epoch : 10/10, Loss: 0.0821, Validation Loss: 0.0859
Time Taken: 321.393274307251


Time Taken: 290

In [82]:

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib as plt
all_preds = []
all_labels = []

model.eval()  # Set model to evaluation mode

  # Disable gradient computation for testing
with torch.inference_mode():    
    for images, labels in test_loader:
        images = images.to(device).float()  # Move images to the appropriate device
        labels = labels.to(device).float()  # Move labels to the appropriate device

        predictions = model(images)  # Get predictions from the model

        # Convert model output (predictions) to class indices
        preds = torch.argmax(predictions, dim=1)
        
        # Convert one-hot encoded labels to class indices
        labels = torch.argmax(labels, dim=1) # add this line for one hot encoded labels
        
        # Store predictions and true labels
        all_preds.extend(preds.cpu().numpy())  # Move to CPU and convert to numpy
        all_labels.extend(labels.cpu().numpy())  # Move to CPU and convert to numpy

# Generate and print the confusion matrix and classification report
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))
print(confusion_matrix(all_labels, all_preds))

#disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(all_labels, all_preds), display_labels=list(range(15)))
#disp.plot(cmap=plt.cm.Blues)
# wandb.finish()

                precision    recall  f1-score   support

      AIM Chat     0.7739    0.5272    0.6272       753
         Email     0.9577    0.9558    0.9568      7176
Facebook Audio     0.9968    0.9953    0.9960     41110
 Facebook Chat     0.8812    0.8651    0.8731      2453
    Gmail Chat     0.8205    0.8743    0.8466      3676
 Hangouts Chat     0.9398    0.9023    0.9207      2958
      ICQ Chat     0.5788    0.8311    0.6824       663
       Netflix     0.9955    0.9969    0.9962     31187
       Spotify     0.9746    0.9584    0.9664      4521
       Youtube     0.9859    0.9827    0.9843      6202

      accuracy                         0.9756    100699
     macro avg     0.8905    0.8889    0.8850    100699
  weighted avg     0.9766    0.9756    0.9757    100699

[[  397    15     1    28    90     8   211     1     0     2]
 [    9  6859    12    26   243     9    16     0     0     2]
 [    0    35 40915    34     8     0     4    41    31    42]
 [   17    16    24  212